# Iterative Prompt Development

Iterative Prompt Development (反復指示開発?) とは、平たくいうとPDCAを回して開発しましょうということ。

なのでGPTに限らずLLMを使う場合は以下を繰り返す。

- Be clear and specific (明確で特定可能にすること)
- Analyze why result does not give desired output (なぜ求めたような回答が得られないのか分析すること)
- Refine the idea and the prompt (アイデアと指示を洗練していく)
- Repeat (繰り返す)

なので、Chat GPTを使った開発を行うには、

- とりあえず何かつくってみよう。

- つくってみた結果を分析しよう。

- 紹介部分を明確にし、もっとAIに考えさせる時間をあげよう。

- 大量の例をもとに指示を洗練させていこう。

## 下準備

In [1]:
f = open("api_key.txt", "r")
api_key = f.read()
#api_key

In [2]:
import openai
openai.api_key=api_key

In [3]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())



In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role":"user", "content":prompt}]
    responce = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0 # modelからの出力のランダム量を調整するパラメータ
    )
    return responce.choices[0].message["content"]

## 実演

製品のファクトシートから説明文を作成してみる

以下の実行結果はただしそうだが、このまま使うには幾つか問題点がある。

ここではその各問題点を順番に解決していく

In [5]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [6]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our stunning mid-century inspired office chair, the perfect addition to any home or business setting. Part of a beautiful family of office furniture, including filing cabinets, desks, bookcases, meeting tables, and more, this chair is available in several options of shell color and base finishes to suit your style. Choose from plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.

The chair is constructed with a 5-wheel plastic coated aluminum base and features a pneumatic chair adjust for easy raise/lower action. It is available with or without armrests and is qualified for contract use. The base finish options are stainless steel, matte black, gloss white, or chrome.

Measuring at a width of 53 cm, depth of 51 cm, and height of 80 cm, with a seat height of 44 cm and seat depth of 41 cm, this chair is designed for ultimate comfort. You can also choose between soft or hard-floor caster options and two choices of seat fo

### Issue1 : 説明文が長すぎる

回答が長すぎる場合、何文字以内、何ワード以内、などの文字、単語数制限をかけることで解決する。

上の例に50単語でと制限を設けたのが以下のもの

これ以外に動画の中では3行で、280文字で、みたいな指定をしている。

三行以内で
Use at most 3 sentences.

280文字以内で
Use at most 280 characters.

In [11]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

print("word num:", len(response.split(" ")))


Introducing our mid-century inspired office chair, perfect for home or business settings. Available in a range of shell colors and base finishes, with or without armrests. Choose from 10 fabric and 6 leather options for full or plastic upholstery. With a 5-wheel base and pneumatic chair adjust, it's both stylish and functional. Made in Italy.
word num: 55


### Issue2 : 要約する際に違う詳細に注目してしまう

商品などであれば、注目してほしいスペックなどがあったりする。

その場合はどのスペックに注目してほしいかを指示することで出力を調整することができる。

例えば下の例でいうと、

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

(これは家具小売店に向けた説明なので、より技術的かつその商品の製造材料などにフォーカスしてほしい)

という文が追加されている。

その次の例では、最後に商品のプロダクトナンバーを書いてほしい、ということで下の文が追加されている。

At the end of the description, include every 7-character 
Product ID in the technical specification.


In [9]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our mid-century inspired office chair, perfect for both home and business settings. With a range of shell colors and base finishes, including stainless steel and matte black, this chair is available with or without armrests. The 5-wheel plastic coated aluminum base and pneumatic chair adjust make it easy to move and adjust to your desired height. Made with high-quality materials, including a cast aluminum shell and HD36 foam seat, this chair is built to last.


In [10]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our mid-century inspired office chair, perfect for home or business settings. With a range of shell colors and base finishes, and the option of plastic or full upholstery, this chair is both stylish and comfortable. Constructed with a 5-wheel plastic coated aluminum base and pneumatic chair adjust, it's also practical. Available with or without armrests and suitable for contract use. Product ID: SWC-100, SWC-110.


### Issue3 : 説明文に表を追加したい

商品の説明文だと各種スペック、サイズなどが表で見れるとより分かりやすい

なので、次の例では以下の二段落が追加されている。

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

(
    
    説明文の後、商品の寸法が分かるように表を追加してください。

    表は2列持つ。

    最初の列では各寸法の名前を入れてください。

    二つ目の列では各計測値をインチのみで入れてください。

)

Give the table the title 'Product Dimensions'.

(
    
    表のタイトルは"製品寸法"としてください

)

最後に、生成した表をhtml形式で吐き出してもらうために、以下のような指示が加わっている。

Format everything as HTML that can be used in a website. 
Place the description in a \<div> element.

(

    これら全てをHTMLで表示できるようにフォーマットを合わせてください。

    説明文は\<div>エレメントで括ってください。
    
)


In [12]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
<h2>Mid-Century Inspired Office Chair</h2>
<p>Introducing our mid-century inspired office chair, part of a beautiful family of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. This chair is available in several options of shell color and base finishes, allowing you to customize it to your liking. You can choose between plastic back and front upholstery or full upholstery in 10 fabric and 6 leather options. The base finish options are stainless steel, matte black, gloss white, or chrome. The chair is also available with or without armrests, making it suitable for both home and business settings. Plus, it's qualified for contract use, so you can trust its durability and quality.</p>
<h3>Construction</h3>
<p>The chair features a 5-wheel plastic coated aluminum base and a pneumatic chair adjust for easy raise/lower action. You can choose between soft or hard-floor caster options and two choices of seat foam densities: medium (1.8 lb/ft3) or 

In [13]:
from IPython.display import display, HTML

In [14]:
display(HTML(response))

Width,"53 cm | 20.87"""
Depth,"51 cm | 20.08"""
Height,"80 cm | 31.50"""
Seat Height,"44 cm | 17.32"""
Seat Depth,"41 cm | 16.14"""
